In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
print('Tensorflow version: ', tf.__version__)

Tensorflow version:  2.13.0


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
df = pd.read_csv("/content/drive/MyDrive/Mini ProjectRisk Msc DS/Main Project/NER/Dataset/NER dataset.csv", encoding='utf-8')
df = df.fillna(method='ffill')
df.head(-5)

,Word,POS,Sentence Id,Tag
0,Chinese,JJ,1,O
1,tech,NN,1,O
2,giant,NN,1,O
3,Alibaba,NNP,1,B-IDENTITY
4,has,VBZ,1,O
...,...,...,...,...
204805,wipers,NNS,6311,O
204806,that,WDT,6311,O
204807,have,VBP,6311,O
204808,been,VBN,6311,O


In [4]:
df['Sentence Id'].nunique(), df.Word.nunique(), df.Tag.nunique()

(6311, 16201, 25)

In [5]:
df.groupby('Tag').size().reset_index(name='counts')

,Tag,counts
0,B-ASSET,127
1,B-ATTACK_TYPE,715
2,B-CAMPAIGN,108
3,B-DATE_TIME,968
4,B-IDENTITY,3972
5,B-INDICATOR,345
6,B-INFRASTRUCTURE,348
7,B-LOCATION,1431
8,B-MALWARE,1111
9,B-THREAT_ACTOR,470


In [6]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(df['Tag'])

LabelEncoder()

In [7]:
df['Enc_tag'] = le.transform(df['Tag'])

In [8]:
class SentenceGetter(object):
    def __init__(self, df):
        self.n_sent = 1
        self.df = df
        agg_func = lambda s: [w for w in s['Word'].values.tolist()]
        self.grouped = self.df.groupby('Sentence Id').apply(agg_func)
        self.sentences = [s for s in self.grouped]

In [9]:
getter = SentenceGetter(df)
sentences = getter.sentences

In [10]:
class POSGetter(object):
    def __init__(self, df):
        self.n_sent = 1
        self.df = df
        agg_func = lambda s: [w for w in s['POS'].values.tolist()]
        self.grouped = self.df.groupby('Sentence Id').apply(agg_func)
        self.sentences = [s for s in self.grouped]

In [11]:
getter = POSGetter(df)
POS_ = getter.sentences

In [12]:
class TagGetter(object):
    def __init__(self, df):
        self.n_sent = 1
        self.df = df
        agg_func = lambda s: [w for w in s['Enc_tag'].values.tolist()]
        self.grouped = self.df.groupby('Sentence Id').apply(agg_func)
        self.sentences = [s for s in self.grouped]

In [13]:
getter = TagGetter(df)
Tags= getter.sentences

In [14]:
data = {'Sentence Id' : df['Sentence Id'].unique(),'Word': sentences,'POS': POS_ ,'Tag': Tags }
df1 = pd.DataFrame(data = data)

In [16]:
from sklearn.model_selection import train_test_split

training_dataset, testing_dataset = train_test_split(df1, test_size=0.2, random_state=2018)

# Conditional Random Fields classifier


In [17]:
!pip install -q tensorflow-addons  # version >= 0.15.0 is required
!pip install -q tensorflow
!pip install -q datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 612.1/612.1 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 20.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.9/294.9 kB 30.5 MB/s eta 0:00:00


In [18]:
import pandas as pd
from datasets import Dataset

def dataframe_to_conll(df):
    conll_lines = []
    for i, row in df.iterrows():
        word = row['Word']
        pos = row['POS']
        ner = row['Tag']
        conll_lines.append(f"{word}{pos}{ner}")

    conll_dataset = '\n'.join(conll_lines)
    return conll_dataset

conll_dataset = dataframe_to_conll(training_dataset)
dataset = Dataset.from_pandas(training_dataset)

In [20]:
import copy
import tensorflow_addons as tfa

/usr/local/lib/python3.10/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


The dataset also give the information about the mapping of NER tags and ids.

In [21]:
raw_tags = df.Tag.unique().tolist()
print(raw_tags)

['O', 'B-IDENTITY', 'B-LOCATION', 'B-TOOL', 'B-DATE_TIME', 'I-IDENTITY', 'I-DATE_TIME', 'B-MALWARE', 'B-ASSET', 'I-ASSET', 'B-ATTACK_TYPE', 'B-VULNERABILITY', 'I-LOCATION', 'B-INFRASTRUCTURE', 'I-MALWARE', 'B-THREAT_ACTOR', 'I-THREAT_ACTOR', 'I-TOOL', 'B-INDICATOR', 'I-INFRASTRUCTURE', 'I-ATTACK_TYPE', 'I-INDICATOR', 'B-CAMPAIGN', 'I-CAMPAIGN', 'I-VULNERABILITY']


Add a special tag `<PAD>` to the tag set which is used to represent a padding in the sequence. In NLP, 0 is usually used to mark padding.

In [22]:
tags = ['<PAD>'] + raw_tags
print(tags)

['<PAD>', 'O', 'B-IDENTITY', 'B-LOCATION', 'B-TOOL', 'B-DATE_TIME', 'I-IDENTITY', 'I-DATE_TIME', 'B-MALWARE', 'B-ASSET', 'I-ASSET', 'B-ATTACK_TYPE', 'B-VULNERABILITY', 'I-LOCATION', 'B-INFRASTRUCTURE', 'I-MALWARE', 'B-THREAT_ACTOR', 'I-THREAT_ACTOR', 'I-TOOL', 'B-INDICATOR', 'I-INFRASTRUCTURE', 'I-ATTACK_TYPE', 'I-INDICATOR', 'B-CAMPAIGN', 'I-CAMPAIGN', 'I-VULNERABILITY']


In [23]:
from sklearn import preprocessing
le2 = preprocessing.LabelEncoder()
le2.fit(tags)

LabelEncoder()

Define some constants which will be used in later.

In [24]:
TAG_SIZE = len(tags)
VOCAB_SIZE = 20000

Building vocabulary lookup layer for tokens.

In [25]:
train_tokens = tf.ragged.constant(dataset['Word'])
train_tokens = tf.map_fn(tf.strings.lower, train_tokens)

lookup_layer = tf.keras.layers.StringLookup(max_tokens=VOCAB_SIZE, mask_token="[MASK]", oov_token="[UNK]")
lookup_layer.adapt(train_tokens)

print(len(lookup_layer.get_vocabulary()))
print(lookup_layer.get_vocabulary()[:10])

12813
['[MASK]', '[UNK]', 'the', ',', '.', 'to', 'and', 'of', 'a', 'in']


Creating raw (without preprocess) train and validation dataset.

In [26]:
def create_data_generator(dataset):
  def data_generator():
    for item in dataset:
      yield item['Word'], item['Tag']

  return data_generator

data_signature= (
        tf.TensorSpec(shape=(None,), dtype=tf.string),
        tf.TensorSpec(shape=(None, ), dtype=tf.int32)
)

train_data = tf.data.Dataset.from_generator(
    create_data_generator(dataset),
    output_signature=data_signature
)

In [27]:
def dataset_preprocess(tokens, tag_ids):
    preprocessed_tokens = preprecess_tokens(tokens)

    # increase by 1 for all tag_ids,
    # because `<PAD>` is added as the first element in tags list
    preprocessed_tag_ids = tag_ids + 1

    return preprocessed_tokens, preprocessed_tag_ids

def preprecess_tokens(tokens):
    tokens = tf.strings.lower(tokens)
    return lookup_layer(tokens)

BATCH_SIZE = 64

# With `padded_batch()`, each batch may have different length
# shape: (batch_size, None)
train_dataset = (
    train_data.map(dataset_preprocess)
    .padded_batch(batch_size=BATCH_SIZE).cache()
)

In [28]:
# Build the model
def build_embedding_bilstm_crf_model(
    vocab_size: int, embed_dims: int, lstm_unit: int, tag_size: int
) -> tf.keras.Model:
    x = tf.keras.layers.Input(shape=(None,), dtype=tf.int32, name="x")
    y = tf.keras.layers.Embedding(vocab_size, embed_dims, mask_zero=True)(x)
    y = tf.keras.layers.Bidirectional(
        tf.keras.layers.LSTM(lstm_unit, return_sequences=True)
    )(y)
    decode_sequence, potentials, sequence_length, kernel = tfa.layers.CRF(tag_size)(y)

    return tf.keras.Model(
        inputs=x, outputs=[decode_sequence, potentials, sequence_length, kernel]
    )


model = build_embedding_bilstm_crf_model(VOCAB_SIZE, 32, 128, TAG_SIZE)

### Define CRF loss function

By using the real y and some internal variables of the CRF layer. You can compute the log likelihood of real y. Use the negative of log likelihood as the loss to optimize.

In [29]:
@tf.function
def crf_loss_func(potentials, sequence_length, kernel, y):
    crf_likelihood, _ = tfa.text.crf_log_likelihood(
        potentials, y, sequence_length, kernel
    )
    # likelihood to loss
    flat_crf_loss = -1 * crf_likelihood
    crf_loss = tf.reduce_mean(flat_crf_loss)

    return crf_loss

### Define optimizer, metrics and train_step fucntion

In [30]:
optimizer = tf.keras.optimizers.Adam(0.02)

train_loss = tf.keras.metrics.Mean(name="train_loss")

@tf.function(experimental_relax_shapes=True)
def train_step(x, y):
    with tf.GradientTape() as tape:
        decoded_sequence, potentials, sequence_length, kernel = model(x)
        crf_loss = crf_loss_func(potentials, sequence_length, kernel, y)
        loss = crf_loss + tf.reduce_sum(model.losses)
    grads = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))

    train_loss(loss)

### Training model

In [31]:
EPOCHS = 40

for epoch in range(EPOCHS):
    # Reset the metrics at the start of the next epoch
    train_loss.reset_states()

    for x, y in train_dataset:
        train_step(x, y)

    print(f"Epoch {epoch + 1}, " f"Loss: {train_loss.result()}")


Epoch 1, Loss: 12.434977531433105
Epoch 2, Loss: 5.096188545227051
Epoch 3, Loss: 3.0153403282165527
Epoch 4, Loss: 2.1791155338287354
Epoch 5, Loss: 1.975490927696228
Epoch 6, Loss: 1.4289493560791016
Epoch 7, Loss: 1.1002627611160278
Epoch 8, Loss: 0.931392252445221
Epoch 9, Loss: 0.9194279909133911
Epoch 10, Loss: 0.7935009002685547
Epoch 11, Loss: 0.5936891436576843
Epoch 12, Loss: 0.4809051752090454
Epoch 13, Loss: 0.39696887135505676
Epoch 14, Loss: 0.3319673538208008
Epoch 15, Loss: 0.30484431982040405
Epoch 16, Loss: 0.25745445489883423
Epoch 17, Loss: 0.26582813262939453
Epoch 18, Loss: 0.32417985796928406
Epoch 19, Loss: 0.30183106660842896
Epoch 20, Loss: 0.318550169467926
Epoch 21, Loss: 0.29097917675971985
Epoch 22, Loss: 0.2657458186149597
Epoch 23, Loss: 0.23021483421325684
Epoch 24, Loss: 0.2472187727689743
Epoch 25, Loss: 0.2243320792913437
Epoch 26, Loss: 0.18950234353542328
Epoch 27, Loss: 0.14031438529491425
Epoch 28, Loss: 0.13747848570346832
Epoch 29, Loss: 0.1333

### Making inference

Inspect the predict result.

In [32]:
# print the inputs and expected outputs
test_text = testing_dataset['Word'].iloc[15]
print("raw inputs: ", test_text)

# preprocess
preprocessed_inputs = preprecess_tokens(
    test_text
)
# expend the batch dim
inputs = tf.reshape(preprocessed_inputs, shape=[1, -1])

outputs, *_ = model.predict(inputs)
prediction = list(le2.inverse_transform(outputs[0]))
ground_true_tags = list(le.inverse_transform(testing_dataset['Tag'].iloc[15]))

print("ground true tags: ", ground_true_tags)
print("predicted tags: ", prediction)

raw inputs:  ['Sadly', 'anyone', 'without', 'proper', 'knowledge', 'or', 'education', 'about', 'phishing', 'links', 'could', 'have', 'fallen', 'victim', 'to', 'this', 'attack', '.']
1/1 [==============================] - 3s 3s/step
ground true tags:  ['O', 'O', 'O', 'O', 'O', 'O', 'B-IDENTITY', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']
predicted tags:  ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']


In [ ]:
y_true = []
y_pred = []
for i in range(len(testing_dataset)):
    # preprocess
    preprocessed_inputs = preprecess_tokens(
        testing_dataset['Word'].iloc[i]
    )
    # expend the batch dim
    inputs = tf.reshape(preprocessed_inputs, shape=[1, -1])

    outputs, *_ = model.predict(inputs)
    prediction = le2.inverse_transform(outputs[0])
    ground_true_tags = le.inverse_transform(testing_dataset['Tag'].iloc[i])
    prediction = list(le.transform(prediction))
    ground_true_tags = list(le.transform(ground_true_tags))
    y_true.extend(ground_true_tags)
    y_pred.extend(prediction)

In [34]:
myset = set(y_true).union(set(y_pred))
lst = list(myset)
labels = list(le.inverse_transform(lst))

In [36]:
y_true = le.inverse_transform(y_true)
y_pred = le.inverse_transform(y_pred)

In [38]:
from sklearn.metrics import classification_report
print(classification_report(y_true, y_pred))

                  precision    recall  f1-score   support

         B-ASSET       0.53      0.43      0.48        23
   B-ATTACK_TYPE       0.74      0.58      0.65        92
      B-CAMPAIGN       0.88      0.68      0.77        22
     B-DATE_TIME       0.84      0.83      0.84       167
      B-IDENTITY       0.71      0.67      0.69       821
     B-INDICATOR       0.39      0.11      0.17       117
B-INFRASTRUCTURE       0.50      0.51      0.50        61
      B-LOCATION       0.85      0.79      0.82       241
       B-MALWARE       0.81      0.81      0.81       183
  B-THREAT_ACTOR       0.63      0.74      0.68        78
          B-TOOL       0.57      0.60      0.58       149
 B-VULNERABILITY       0.50      0.38      0.43        13
         I-ASSET       0.80      0.55      0.65        22
   I-ATTACK_TYPE       0.65      0.43      0.52        35
      I-CAMPAIGN       0.86      0.95      0.90        20
     I-DATE_TIME       0.82      0.78      0.80        77
      I-IDENT

In [39]:
p = []
t = []
for i in y_pred:
  if i == 'O':
    p.append(i)
  else:
    p.append(i[2:])

for i in y_true:
  if i == 'O':
    t.append(i)
  else:
    t.append(i[2:])

In [40]:
from sklearn.metrics import classification_report
print(classification_report(t, p))

                precision    recall  f1-score   support

         ASSET       0.65      0.49      0.56        45
   ATTACK_TYPE       0.73      0.54      0.62       127
      CAMPAIGN       0.87      0.81      0.84        42
     DATE_TIME       0.90      0.88      0.89       244
      IDENTITY       0.73      0.66      0.69      1256
     INDICATOR       0.49      0.21      0.30       127
INFRASTRUCTURE       0.58      0.56      0.57        77
      LOCATION       0.85      0.79      0.82       286
       MALWARE       0.81      0.83      0.82       210
             O       0.98      0.98      0.98     35809
  THREAT_ACTOR       0.65      0.73      0.69       104
          TOOL       0.55      0.52      0.53       187
 VULNERABILITY       0.50      0.50      0.50        26

      accuracy                           0.96     38540
     macro avg       0.71      0.65      0.68     38540
  weighted avg       0.96      0.96      0.96     38540



In [41]:
report = classification_report(t, p, digits=4, output_dict=True)
# Access the weighted F1 score, recall, and precision
f1_weighted = report['weighted avg']['f1-score']

recall_weighted = report['weighted avg']['recall']
precision_weighted = report['weighted avg']['precision']

# Print the results

print ('Weighted F1 Score: ', f1_weighted)
print ('Weighted Recall: ', recall_weighted)
print ('Weighted Precision: ', precision_weighted)

Weighted F1 Score:  0.9609837132610887
Weighted Recall:  0.9623248572911262
Weighted Precision:  0.9602804223080614


In [42]:
report = classification_report(t, p, digits=4, output_dict=True)
# Access the weighted F1 score, recall, and precision
f1_weighted = report['macro avg']['f1-score']

recall_weighted = report['macro avg']['recall']
precision_weighted = report['macro avg']['precision']

# Print the results

print ('Macro F1 Score: ', f1_weighted)
print ('Macro Recall: ', recall_weighted)
print ('Macro Precision: ', precision_weighted)

Macro F1 Score:  0.6775774325868544
Macro Recall:  0.6545587213555485
Macro Precision:  0.7138611207951536


# Case Study

In [43]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
stop_words = set(stopwords.words('english'))

from nltk.tokenize import word_tokenize, sent_tokenize

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [44]:
test_text = """
Although the initial vector of compromise is unclear, the details of the next stage imply the use of spear phishing or similar methods.
 """

In [45]:
Case = word_tokenize(test_text)

preprocessed_inputs = preprecess_tokens(
    Case
)

inputs = tf.reshape(preprocessed_inputs, shape=[1, -1])

outputs, *_ = model.predict(inputs)
prediction = list(le2.inverse_transform(outputs[0]))

for token, label in zip(Case, prediction):
    print("{:20}\t{}".format(token, label))

1/1 [==============================] - 0s 58ms/step
Although            	O
the                 	O
initial             	O
vector              	O
of                  	O
compromise          	O
is                  	O
unclear             	O
,                   	O
the                 	O
details             	O
of                  	O
the                 	O
next                	O
stage               	O
imply               	O
the                 	O
use                 	O
of                  	O
spear               	B-ATTACK_TYPE
phishing            	I-ATTACK_TYPE
or                  	O
similar             	O
methods             	O
.                   	O
